# FM06 Mock Coursework

## Before you begin

Every student has a slightly different assignment. To generate your personal
assignment you should first execute every cell in this Jupyter notebook (which you can do by clicking Validate).
You must then answer the questions given in the notebook.

The first cell in this Notebook generates your question for you.

You must complete this assignment by editing the Jupyter notebook distributed to you in your Assignments folder on CoCalc.

The Jupyter notebook containing your work will be automatically collected after the submission deadline and this will be used to automatically mark your numerical answers.
Your mathematical writing will also be marked, but by one of your colleagues. I will explain the mark scheme once the assignment is completed.

Although I am calling this a "Mock Coursework" assignment, the question asked is somewhat shorter than the actual coursework assignment. Another difference is that, in your final coursework
you should not present your answer as a Jupyter notebook. Instead you will be asked to write a standalone LaTeX document. However, many of the skills I am looking for are the same
in this assignment and your final coursework.

In [3]:
import pandas
import numpy as np
from math import *
import coursework1lib

question = coursework1lib.Question()


## Reading stock price data from a file

The file `stock-data.csv` contains weekly historic stock prices for two companies *ACME* and *BigBank*. All prices are in dollars. At each historic time point $t$,
you can represent the stock prices as a vector ${\mathbf S}_t$ whose first component is the price of *ACME* and whose second component
is the price of *BigBank*. The logs of the stock prices can similarly be represented as vector $Z_t$. Let $\delta t=\frac{1}{52}$ be the time
interval of 1 week (so for the purposes of this question 1 year contains exactly 52 weeks and we are using the
financial convention of using the year as our unit of time). Write $(\delta Z)_t=Z_{t+\delta t}-Z_t$.

Read the file `stock-data.csv` into Python and use this to do the following:

* Create a matrix called `dZ` with $N-1$ rows and $2$ columns containing all the weekly changes in the log stock price. Here $N$ is the number of weeks in the sample.
* The sample average value of the vector $(\delta Z)_t$. Store the value as a `numpy` 2-vector in a variable `m`. We will refer to this vector in the mathematics below as $\bm{m}$.
* The sample covariance matrix for the change $(\delta Z)_t$. Store the value as a `numpy` array in a variable `Sigma`. We will refer to this matrix in the mathematics below as $\Sigma$.

We have not seen how to read a `csv` file in the course. You can do this using `pandas.read_csv`, or you can use Excel to save the file in the Excel format that we have seen how to read, or you can use the Python library `csv`, or you could use some other approach. It is entirely up to you.

In [4]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [5]:
assert m.shape==(2,)
assert Sigma.shape==(2,2)

NameError: name 'm' is not defined

## Simulating stock prices

Assume that over future weekly intervals, the changes in the log stock prices $(\delta Z)_t$ will follow a multivariate normal distribution with
mean $\bm{m}$ and covariance matrix $\Sigma$. You should assume that each week's increments are independent of each other. Write $t=0$ for the final time in the historic data so that $S_0$ give the stock prices at the final time in the historic data. Generate $10^6$ simulated stock price vectors
prices at time $t=1$. You should store your simulation in a variable called `simulated_S` which should be a `numpy` array
with $10^6$ rows and 2 columns.

You must also explain mathematically how you performed this computation. You can do this by giving mathematical difference equations that show how the computation was performed together with appropriate explanatory text. Your mark will be based in part on the quality of the formatting of the mathematical formulae.

**Solution:** *Insert your mathematical explanation here. Your mark will be based in part upon the quality of the formatting of the mathematical formulae.*

In [6]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [7]:
n_scenarios = 1000000
n_steps = 52
assert simulated_S.shape==(n_scenarios,2)

NameError: name 'simulated_S' is not defined

## Computing option prices

At time $0$, a trader purchases assets as described below and holds the assets for 52 weeks.
To purchases these assets they must borrow money at the interest rate specified below.

In [8]:
question.describe_portfolio()

* 3 units of stock 1.
* 7 European call options on stock 1 with strike 1625 and maturity 52 weeks.
* 3 European put options on stock 2 with strike 824 and maturity 52 weeks.
* 9 European call options on stock 2 with strike 919 and maturity 52 weeks.


The continuously compounded interest rate is r=0.04.

A European call option with strike $K$ and maturity $T$ on a stock is a derivative which gives a payout of $\max\{S_T-K,0\}$ at the maturity, where $S_T$ is the price of the stock time $T$. A European put option with strike $K$ and maturity $T$ gives 
a payout of $\max\{K-S_T,0\}$ at maturity.

You should assume that the options are purchased at the *Black-Scholes price*. For a call option with strike $K$ and maturity $T$,
on a stock with price $S_0$ at time $0$, this is computed using the formula
$$
\text{Call option price}=N(d_1)S_0 - Ke^{-rT}N(d_2)
$$
where
$$
d_1 = \frac{1}{\sigma \sqrt{T}}\left( \log(S_0/K) + (r+\frac{\sigma^2}{2})T\right)
$$
and
$$
d_2 = \frac{1}{\sigma \sqrt{T}}\left( \log(S_0/K) + (r-\frac{\sigma^2}{2})T\right).
$$
Here $N$ is the cumulative distribution function of the standard normal distribution,
and $\sigma$ is a parameter called the volatility. For stock $i$, you should assume
$\sigma=\sqrt{52 \, \Sigma_{i,i}}$ where $\Sigma$ is the $1$-week covariance matrix of the changes in the log
stock price. 

This formula will be explained later in the course. For now you simply need to write a Python function which can compute it. Since
you do not yet know the theory used to derived this formula, I have provided some tests that you can use to check your have
implemented this formula correctly.

In [9]:
def black_scholes_call_price(S0,K,T,r,sigma):
    # YOUR CODE HERE
    raise NotImplementedError()

In [10]:
np.testing.assert_almost_equal( black_scholes_call_price( 100, 110, 0.5, 0.1, 0.2), 3.743, decimal=3)

NotImplementedError: 

The Black-Scholes price formula for a European put option with strike $K$ and maturity $T$ is
$$
e^{-rT} K N(-d_2) - N(-d_1) S_0
$$
Write a function to compute this.

In [11]:
def black_scholes_put_price(S0,K,T,r,sigma):
    # YOUR CODE HERE
    raise NotImplementedError()

In [12]:
np.testing.assert_almost_equal( black_scholes_put_price( 100, 110, 0.5, 0.1, 0.2), 8.378, decimal=3)

NotImplementedError: 

Compute the total cost of the portfolio and store the answer in a variable called `cost`

In [13]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

## Historic volatility

The value $\sqrt{52 \, \Sigma_{i,i}}$ is called the *historic volatility* of stock $i$. The factor of $52$ in the formula for the historic volatility is a scaling factor to take account of the fact that one usually uses units of a year in financial calculations. The square root in the formula comes from the fact that $\Sigma_{i,i}$ is the one-week variance of stock $i$, and so its square root is the one-week standard deviation.

There are many assumptions required
to derive the Black-Scholes price formula including assumptions on the *future* volatility of the stock. The parameter $\sigma$ in
the Black-Scholes formulae represents this future volatility. In your calculation you have been told to assume that the future volatility will be equal to the historic volatility, but this is a very unrealistic assumption.

There are often
very good reasons to think that future volatility will be different from past volatility. For example, as I write this question there is an election looming in the United States, so one might believe that volatility of the US stock market will be high as the election approaches and will drop after the election when the outcome is known and so there is less political uncertainty. In fact, volatility is much easier to predict than whether a stock will go up or down. This is one important reason why real option prices will often be very different from those obtained using the Black-Scholes formula and historic volatility. 

## Compute the Value at Risk and Expected Shortfall

After $1$ year, the trader liquidates their portfolio and pays of their debt with interest. Compute the
$95$-th percentile of the loss distribution.
This gives an estimate of a risk-figure which is called the 52 week, $5\%$ *Value at Risk*
of the portfolio. Store your answer in a variable called `var`.

Compute the average value of the loss in all the scenarios where the loss was greater than the Value at Risk.
This gives an estimate of another risk-figure which called the 52 week, %5\$ *Expected Shortfall* of the portfolio.
Store your answer in a variable called `es`.

Value at Risk and Expected Shortfall figures are often used in the industry and can be computed over a variety of time-horizons
and at different percentage levels.

In [14]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [15]:
assert es>=var

NameError: name 'es' is not defined

## Testing your code

Your code will be tested automatically as part of the marking process. However, you should write your own tests and explain how they work.
The better your tests, the higher your mark will be. For each test that you write, provide a mathematical description of the test as well
as the code that performs the test.

YOUR ANSWER HERE

## Discussion

Write a short essay on the topic of risk measurement. For example you could discuss: the history and purpose of different risk measures including Value at Risk and Expected Shortfall; the relative merits of different risk alternative approaches to risk-measurement; any limitations of the calculations you have performed for this exercise. Your essay should take up at most one
page when the notebook is formatted as a pdf. Use File->Save and Download as->PDF to view the notebook as a PDF.

Be sure to do the following:

* Reference scholarly works to justify any claims that you make.
* Reference any sources you have used (for example if you have read an article on the internet, you should reference it)
* Use your own words

It is vital to always reference your sources properly and to use your own words otherwise you may be accused of *plagiarism*.
This is a serious academic offence. In exams it is acceptable to use the lecturer's own words when answering a question but
in your dissertation and in this coursework you must use your own words. You can find more information on referencing and plagiarism on Keats.

I am asking you to write a short essay because you will eventually have to write a long dissertation so it is good to start
practicing writing about mathematics. I would give the following general advice:

* Try to write something *you* would enjoy reading.
* Don't try to impress the reader with fancy words, impress them with *facts* and *mathematics*.
* Don't say anything you don't understand yourself.
* Say things as clearly and briefly as you can.
* If you find it easier to say things with mathematical formulae than in English, then focus on the mathematical content.
* Don't pad what you have to say with *waffle* just to reach the word limit or to try to look more impressive.

YOUR ANSWER HERE

## Feedback

Your work will be automatically marked by cocalc to answer the numerical sections so long as it is submitted on time.

### Are you happy for me to discuss your solutions in the skills session?

I will look at some solutions and may discuss them in a skills session without identifying whose solutions they are. Please enter the string "Yes" below if you are
happy for me to discuss your solution in the skills session. Please enter the string "No" otherwise.



In [16]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [17]:
if _=="Yes" or _=="No":
    pass
else:
    raise Exception('You must type either "Yes" or "No"')

Exception: You must type either "Yes" or "No"